In [1]:
import sys
sys.path.append('../')

import numpy
from sklearn.neighbors import (NeighborhoodComponentsAnalysis,KNeighborsClassifier)
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

from keras.layers import SimpleRNN
from keras.models import Sequential

from testboard.data_mining.helpers.stock_util import StockUtil
from testboard.data_mining.smote import duplicate_data
from testboard.models.densegru import DenseGRU
from testboard.models.denselstm import DenseLSTM

Using TensorFlow backend.


### Getting stock data

In [2]:
stockutil = StockUtil(['PETR3'], [9])
prices, preds = stockutil.average_prices_preds()

### Slice dataset in training data and test data

In [3]:
train_proportion = 0.7

train_size = int(len(prices[0]) * train_proportion)

train_x = prices[0][0:train_size]
test_x = prices[0][train_size:len(prices[0])]
train_y = preds[0][0:train_size]
test_y = preds[0][train_size:len(prices[0])]

### Testing on LSTM

In [4]:
lstm = DenseLSTM(look_back=6, dense=False)
lstm.create_data_for_fit(numpy.array(prices[0]).reshape(len(prices[0]), 1))
result = lstm.fit_and_evaluate(batch_size=1, epochs=5000)
print("accuracy: " + str(result['acc']))

KeyboardInterrupt: 

### Testing on KNN

In [ ]:
X = numpy.array(train_x).reshape(len(train_x), 1)
print(X)
Y = train_y

neigh = KNeighborsRegressor(n_neighbors=5)
neigh.fit(X, Y)
X_test = numpy.array(test_x).reshape(len(test_x), 1)

print(neigh.predict(X_test))
score = 0
for i, pred in enumerate(neigh.predict(X_test)):
    if (pred >= 0.5 and test_y[i] is 1) or (pred < 0.5 and test_y[i] is 0):
        score += 1
        
print("accuracy: " + str(score/len(test_y)))

### Testing on LSTM optimized

In [ ]:
lstm = DenseLSTM(look_back=6, dense=1, lstm_cells=50)
lstm.create_data_for_fit(numpy.array(prices[0]).reshape(len(prices[0]), 1))
result = lstm.fit_and_evaluate(batch_size=1, epochs=5000)
print("accuracy: " + str(result['acc']))

### Testing on GRU

In [ ]:
gru = DenseGRU(look_back=6, dense=False)
gru.create_data_for_fit(numpy.array(prices[0]).reshape(len(prices[0]), 1))
result = gru.fit_and_evaluate(batch_size=1, epochs=5000)
print("accuracy: " + str(result['acc']))

### Testing on SVM

In [ ]:
svr = SVR(kernel='linear', C=1, epsilon=0.2)
svr.fit(X, Y)
X_test = numpy.array(test_x).reshape(len(test_x), 1)
svr.predict(X_test)

### Testing on RNN

In [ ]:
model = Sequential()

model.add(SimpleRNN(activation="sigmoid", units=1))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(gru.train_x, gru.train_y, epochs=5000, verbose=0)
result = model.evaluate(gru.test_x, gru.test_y)
print('accuracy: ' + str(result[1]))